In [ ]:
from src.harness import architecture as arch
from src.harness import evolution as evo
from src.harness import utils

import copy
from enum import Enum
import functools
from importlib import reload
import itertools
from matplotlib import pyplot as plt
import multiprocess as mp
import numpy as np
import tensorflow as tf
from tensorflow import keras
from typing import Any, Callable, Dict, Iterable, List, Literal, Set, Tuple

reload(arch)
reload(evo)

In [ ]:
architecture = arch.Architecture("lenet", "mnist")

model_features = [
    evo.ModelFeatures.layer_sparsity, 
    evo.ModelFeatures.magnitude,
    evo.ModelFeatures.random,
    functools.partial(evo.ModelFeatures.synaptic_flow, loss_fn=keras.losses.CategoricalCrossentropy()),
]
arch_feature_selectors = [
    evo.ArchFeatures.layer_num,
    evo.ArchFeatures.layer_ohe,
    evo.ArchFeatures.layer_prop_params,
]

individual = evo.Individual("lenet", "mnist", model_features, arch_feature_selectors, [])

In [ ]:
num_runs = 3

model_feature_selectors = [
    evo.ModelFeatures.layer_sparsity, 
    evo.ModelFeatures.magnitude,
    evo.ModelFeatures.random,
    functools.partial(evo.ModelFeatures.synaptic_flow, loss_fn=keras.losses.CategoricalCrossentropy()),
]
arch_feature_selectors = [
    evo.ArchFeatures.layer_num,
    evo.ArchFeatures.layer_ohe,
    evo.ArchFeatures.layer_prop_params,
]

layers = []

individual_constructor = functools.partial(
    evo.Individual, 
    architecture_name="lenet",
    dataset_name="mnist",
    model_feature_selectors=model_feature_selectors,
    arch_feature_selectors=arch_feature_selectors,
    layers=layers,
)

objectives = [
    (evo.Target.MAXIMIZE, lambda x: 1, evo.Individual.eval_accuracy),
    (evo.Target.MINIMIZE, lambda x: 1, evo.Individual.sparsity),
]

rate_func = lambda n: 0.5
scale_func = lambda n: 1 / np.sqrt(n + 1)
mutations = [
    functools.partial(evo.Individual.get_annealing_mutate(), rate=rate_func, scale=scale_func),
    evo.Individual.update_phenotype,
]
    
genome_metric_callbacks = [
]
kwargs = {
    "num_generations": 10,
    "archive_size": 10,
    "population_size": 10,
    "fronts_to_consider": 2,
    "tournament_size": 4,
    "num_tournament_winners": 2,
    "individual_constructor": individual_constructor,
    "objectives": objectives,
    "mutations": mutations,
    "crossover": evo.Individual.crossover,
    "genome_metric_callbacks": genome_metric_callbacks,
}

all_genome_metrics = []
all_objective_metrics = []
all_archives = []
for run in range(num_runs):
    print(f"Run {run + 1}")
    genome_metrics, objective_metrics, archive = evo.nsga2(**kwargs)
    all_genome_metrics.append(genome_metrics)
    all_objective_metrics.append(objective_metrics)
    all_archives.append(archive)

In [ ]:
all_objective_metrics